In [1]:
#https://stackoverflow.com/questions/47944782/r-data-scraping-crawling-with-dynamic-multiple-urls
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

ModuleNotFoundError: No module named 'requests'

In [ ]:
import locale, datetime #for stringparsing in german
locale.setlocale(locale.LC_ALL, "de_de")

**Extracting the index leading to mandate**

In [ ]:
#highlight_docid=atf%3A%2F%2F95-IV-162
queryKeyword = "mord"
queryRank = "1"
#bgerQuery2 = "https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=1954&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words="+ queryKeyword + "&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank="+queryRank+"&highlight_docid=atf%3A%2F%2F95-IV-162%3Ade&number_of_ranks=945&azaclir=clir"

In [ ]:
bgerQuery2 = 'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words='+ queryKeyword +'&lang=de&top_subcollection_clir=bge&from_year=1954&to_year=2019'

In [ ]:
bgerQuery2

In [ ]:
#check if content was loaded properly
bgerQuery2 = requests.get(bgerQuery2)
bgerQuery2

In [ ]:
#bgerQuery2.content

In [ ]:
#soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
bgerSoup = BeautifulSoup(bgerQuery2.content, 'html.parser')

In [ ]:
bgerSoup

In [ ]:
links = bgerSoup.find_all(class_='rank_title')
linksList = []
for a in links:
    linksList.append(a.find('a')['href'])
    #print(a)
print(linksList[0])

**Make a new query with Linkslist (only based on first entry)**

In [ ]:
mBger = requests.get(linksList[0])

In [ ]:
#mBger.content

In [ ]:
mBger = BeautifulSoup(mBger.content, 'html.parser')

In [ ]:
#mBger

**display children / find correct html elements**

In [ ]:
len(list(mBger.children))
#find body
body = list(mBger.children)[12]
len(list(body.children))
#body

**get all required variables from verdict**

**dateparser for string dates**

In [ ]:
text2 = "41. Urteil des Kassationshofes vom 5. Dezember 1969 i.S. Staatsanwaltschaft des Kantons Aargau gegen Brunner."

def parseDate(text):
    pattern = re.compile(r"""
        \d\d?   # one or two digits
        \.?
        \s?      # \s for space - one space after
        [a-z,ä,ö,ü]+  # at least one+ ascii letters (ignore case is use)set of characters that you wish to match / since + means ‘one or more repetitions’
        \s?      # one space after
        \d{4}   # four decimal digits = \d (year)
    """,re.IGNORECASE|re.VERBOSE)
    if (pattern.search(text)!= None):
        #print(pattern.search(text).group(), type(pattern.search(text).group()))
        dateStr = pattern.search(text).group()
        dateStr = dateStr.replace(".", "")
        dateStr = datetime.datetime.strptime(dateStr, "%d %B %Y")
        #dateStr = dateStr.strftime('%m/%d/%Y')
        return dateStr
    else:
        return 0 #make 0 if date can't be found

In [ ]:
#ID of mandate
mID = body.find_all('div', class_="center pagebreak")[0].get_text()[:13]
#mandID

#referenced mandates
#without link
mRefs = body.find_all(class_="bgeref_err")
#with link
mRefsLinked = body.find_all(class_="bgeref_id")

#date:
mDate = body.find_all(class_="paraatf")[0].get_text()
#mDate = parseDate(mDate)
mDate = parseDate(mDate)
mDate

#links of liked reference mandates
linksListref = []
for a in mRefsLinked:
    linksListref.append(a['href'])

In [ ]:
type(mDate)

In [ ]:
def listing(elementlist):
    ar = []
    for item in elementlist:
        print(item.get_text())
        ar.append(item.get_text())
    print(ar)
    return ar

In [ ]:
mRefs = listing(mRefs)
mRefsLinked = listing(mRefsLinked)

In [ ]:
"id", mID, "referenced mandates", mRefs, "referenced linked mandates", mRefsLinked, "date", mDate

**Save variables in dynamic dictionary**

In [ ]:
l=['mID','mRefs','mRefsLinked','links','date']
objs = [mID,mRefs, mRefsLinked, linksListref, mDate]
mDict = {}
for i, val in enumerate(l):
    k= objs[i]
    mDict[val]=k
print(mDict, type(mDict))

In [ ]:
#https://stackoverflow.com/questions/13575090/construct-pandas-dataframe-from-items-in-nested-dictionary
#pd.DataFrame.from_dict(mDict, orient='index')